In [1]:
import os
import time
from datetime import datetime, timezone

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from aipricepatterns import Client

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 160)

def safe_float(x, default=0.0) -> float:
    try:
        return float(x)
    except Exception:
        return float(default)

def map_suggested_action_to_pos(x) -> int:
    if x is None:
        return 0
    if isinstance(x, (int, float)):
        v = int(x)
        if v in (-1, 0, 1):
            return int(v)
        if v in (0, 1, 2):
            return 1 if v == 1 else (-1 if v == 2 else 0)
        return 0
    if not isinstance(x, str):
        return 0
    s = x.strip().lower()
    if s in ('hold', 'flat', 'none', 'neutral', 'wait'):
        return 0
    if s in ('long', 'buy', 'bull', 'up'):
        return 1
    if s in ('short', 'sell', 'bear', 'down'):
        return -1
    return 0

def episode_policy_pnl(ep: dict, horizon: int, trade_cost_pct: float, policy: str, conf_thr: float) -> float:
    ts = ep.get('transitions')
    if not isinstance(ts, list) or not ts:
        return 0.0
    sim = safe_float(ep.get('similarity'), 0.0)
    steps = min(len(ts), int(horizon))
    pos = 0
    pnl = 0.0
    cost = 0.0
    for i in range(steps):
        t = ts[i] if isinstance(ts[i], dict) else {}
        ret = safe_float(t.get('ret', t.get('return', 0.0)), 0.0)
        old_pos = pos

        if policy == 'alwaysFlat':
            pos = 0
        elif policy == 'alwaysLong':
            pos = 1
        elif policy == 'alwaysShort':
            pos = -1
        elif policy == 'suggested':
            pos = map_suggested_action_to_pos(t.get('suggestedAction'))
        elif policy == 'suggestedIfConf':
            pos = map_suggested_action_to_pos(t.get('suggestedAction')) if sim >= conf_thr else 0
        else:
            pos = 0

        if pos != old_pos and trade_cost_pct != 0.0:
            cost += abs(pos - old_pos) * (trade_cost_pct / 100.0)

        pnl += float(pos) * ret

    return float(pnl - cost)

## Parameters
We evaluate over **300 anchors** on BTCUSDT 1h by default.
Tune `EPISODES_PER_ANCHOR` down if you want faster iterations.

In [2]:
BASE_URL = os.getenv('AIPP_BASE_URL', 'https://aipricepatterns.com/api/rust')
API_KEY = os.getenv('AIPP_API_KEY')

SYMBOL = os.getenv('AIPP_RL_SYMBOL', 'BTCUSDT')
INTERVAL = os.getenv('AIPP_RL_INTERVAL', '1h')

ANCHOR_POINTS = int(os.getenv('AIPP_SWEEP_ANCHORS', '300'))
LOOKBACK_DAYS = int(os.getenv('AIPP_SWEEP_LOOKBACK_DAYS', '120'))

FORECAST_HORIZON = int(os.getenv('AIPP_RL_HORIZON', '24'))
EPISODES_PER_ANCHOR = int(os.getenv('AIPP_SWEEP_EPISODES_PER_ANCHOR', '100'))
MIN_SIMILARITY = float(os.getenv('AIPP_RL_MIN_SIMILARITY', '0.70'))
SAMPLING_STRATEGY = os.getenv('AIPP_RL_SAMPLING_STRATEGY', 'uniform')

TRADE_COST_PCT = float(os.getenv('AIPP_RL_TRADE_COST_PCT', '0.00'))
CONF_THR = float(os.getenv('AIPP_RL_SUGGESTED_MIN_SIMILARITY', '0.90'))

SWEEP_SLEEP_SEC = float(os.getenv('AIPP_SWEEP_SLEEP_SEC', '0.10'))

CACHE_DIR = os.getenv('AIPP_RESEARCH_CACHE_DIR', 'python-sdk/research/_cache')
CACHE_PATH = os.path.join(CACHE_DIR, f'04_retrieval_walkforward_{SYMBOL}_{INTERVAL}_{ANCHOR_POINTS}.csv')

print('Base URL:', BASE_URL)
print(f'Symbol: {SYMBOL}  Interval: {INTERVAL}')
print(f'Anchors: {ANCHOR_POINTS}  LookbackDays: {LOOKBACK_DAYS}')
print(f'Episodes/anchor: {EPISODES_PER_ANCHOR}  minSimilarity: {MIN_SIMILARITY:.2f}  horizon: {FORECAST_HORIZON}')
print(f'TradeCostPct: {TRADE_COST_PCT:.4f}%  ConfThr: {CONF_THR:.2f}')
print('Cache:', CACHE_PATH)

Base URL: https://aipricepatterns.com/api/rust
Symbol: BTCUSDT  Interval: 1h
Anchors: 300  LookbackDays: 120
Episodes/anchor: 100  minSimilarity: 0.70  horizon: 24
TradeCostPct: 0.0000%  ConfThr: 0.90
Cache: python-sdk/research/_cache/04_retrieval_walkforward_BTCUSDT_1h_300.csv


## Walk-forward evaluation (with caching)
For each anchor we compute expected horizon PnL under policies on the retrieved episode distribution:
- alwaysFlat / alwaysLong / alwaysShort
- suggested
- suggestedIfConf (flat if episode similarity < CONF_THR)

In [3]:
os.makedirs(CACHE_DIR, exist_ok=True)

now_ms = int(time.time() * 1000)
start_ms = now_ms - LOOKBACK_DAYS * 24 * 60 * 60 * 1000
anchors = np.linspace(start_ms, now_ms, num=ANCHOR_POINTS, dtype=np.int64).tolist()

if os.path.exists(CACHE_PATH):
    wf = pd.read_csv(CACHE_PATH)
    print('loaded cache:', CACHE_PATH, 'rows:', len(wf))
else:
    client = Client(base_url=BASE_URL, api_key=API_KEY)

    rows = []
    for idx, anchor_ts in enumerate(anchors, start=1):
        res = client.get_rl_episodes(
            symbol=SYMBOL,
            interval=INTERVAL,
            anchor_ts=int(anchor_ts),
            forecast_horizon=FORECAST_HORIZON,
            num_episodes=EPISODES_PER_ANCHOR,
            min_similarity=MIN_SIMILARITY,
            include_actions=True,
            reward_type='returns',
            sampling_strategy=SAMPLING_STRATEGY,
        )
        eps = res.get('episodes') if isinstance(res, dict) else None
        if not isinstance(eps, list) or not eps:
            rows.append({
                'anchorTs': int(anchor_ts),
                'anchorDtUtc': datetime.fromtimestamp(anchor_ts/1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M'),
                'episodes': 0,
                'meanSimilarity': np.nan,
                'avg_flat': np.nan,
                'avg_long': np.nan,
                'avg_short': np.nan,
                'avg_suggested': np.nan,
                'avg_suggestedIfConf': np.nan,
            })
            continue

        sim = np.array([safe_float(ep.get('similarity'), np.nan) for ep in eps], dtype=float)
        mean_sim = float(np.nanmean(sim))

        pnls_flat = [episode_policy_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, 'alwaysFlat', CONF_THR) for ep in eps]
        pnls_long = [episode_policy_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, 'alwaysLong', CONF_THR) for ep in eps]
        pnls_short = [episode_policy_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, 'alwaysShort', CONF_THR) for ep in eps]
        pnls_sug = [episode_policy_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, 'suggested', CONF_THR) for ep in eps]
        pnls_conf = [episode_policy_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, 'suggestedIfConf', CONF_THR) for ep in eps]

        rows.append({
            'anchorTs': int(anchor_ts),
            'anchorDtUtc': datetime.fromtimestamp(anchor_ts/1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M'),
            'episodes': int(len(eps)),
            'meanSimilarity': mean_sim,
            'avg_flat': float(np.mean(pnls_flat)),
            'avg_long': float(np.mean(pnls_long)),
            'avg_short': float(np.mean(pnls_short)),
            'avg_suggested': float(np.mean(pnls_sug)),
            'avg_suggestedIfConf': float(np.mean(pnls_conf)),
        })

        if idx % 10 == 0:
            print(f'{idx}/{len(anchors)} anchors...')
        time.sleep(SWEEP_SLEEP_SEC)

    wf = pd.DataFrame(rows)
    wf.to_csv(CACHE_PATH, index=False)
    print('wrote cache:', CACHE_PATH, 'rows:', len(wf))

wf.head()

10/300 anchors...
20/300 anchors...
30/300 anchors...
40/300 anchors...
50/300 anchors...
60/300 anchors...
70/300 anchors...
80/300 anchors...
90/300 anchors...
100/300 anchors...
110/300 anchors...
120/300 anchors...
130/300 anchors...
140/300 anchors...
150/300 anchors...
160/300 anchors...
170/300 anchors...
180/300 anchors...
190/300 anchors...
200/300 anchors...
210/300 anchors...
220/300 anchors...
230/300 anchors...
240/300 anchors...
250/300 anchors...
260/300 anchors...
270/300 anchors...
280/300 anchors...
290/300 anchors...
300/300 anchors...
wrote cache: python-sdk/research/_cache/04_retrieval_walkforward_BTCUSDT_1h_300.csv rows: 300


,anchorTs,anchorDtUtc,episodes,meanSimilarity,avg_flat,avg_long,avg_short,avg_suggested,avg_suggestedIfConf
0,1755726570584,2025-08-20 21:49,100,0.940809,0.0,0.248794,-0.248794,3.977389,3.977389
1,1755761246169,2025-08-21 07:27,100,0.899479,0.0,0.127091,-0.127091,5.414005,1.542120
2,1755795921754,2025-08-21 17:05,100,0.897753,0.0,-0.107663,0.107663,8.694409,2.839264
3,1755830597339,2025-08-22 02:43,100,0.941799,0.0,0.297466,-0.297466,9.802987,9.802987
4,1755865272925,2025-08-22 12:21,100,0.957717,0.0,0.015437,-0.015437,7.821880,7.821880


## Results
We plot the rolling expected PnL (per-anchor) and the cumulative sum (as a visual proxy).

In [4]:
wf_clean = wf.dropna(subset=['avg_suggested', 'avg_suggestedIfConf', 'avg_long', 'avg_short', 'avg_flat']).copy()
print('valid anchors:', len(wf_clean), '/', len(wf))

for col in ['avg_flat', 'avg_long', 'avg_short', 'avg_suggested', 'avg_suggestedIfConf']:
    wf_clean[f'cum_{col}'] = wf_clean[col].cumsum()

wf_clean[['meanSimilarity','avg_flat','avg_long','avg_short','avg_suggested','avg_suggestedIfConf']].describe().T

valid anchors: 300 / 300


,count,mean,std,min,25%,50%,75%,max
meanSimilarity,300.0,0.928956,0.046067,0.738254,0.906584,0.940867,0.965944,0.986072
avg_flat,300.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
avg_long,300.0,0.079718,0.518785,-1.319217,-0.295111,0.079877,0.419882,2.148344
avg_short,300.0,-0.079718,0.518785,-2.148344,-0.419882,-0.079877,0.295111,1.319217
avg_suggested,300.0,7.179583,2.262531,3.117410,5.462801,6.761523,8.319640,16.963382
avg_suggestedIfConf,300.0,5.805482,3.575762,0.000000,3.725246,6.131074,8.099972,16.963382


In [5]:
fig = go.Figure()
for col, name in [
    ('avg_flat', 'alwaysFlat'),
    ('avg_long', 'alwaysLong'),
    ('avg_short', 'alwaysShort'),
    ('avg_suggested', 'suggested'),
    ('avg_suggestedIfConf', 'suggestedIfConf'),
]:
    fig.add_trace(go.Scatter(x=wf_clean['anchorTs'], y=wf_clean[col], mode='lines', name=name))
fig.update_layout(title='Per-anchor expected horizon PnL (retrieved episode distribution)', xaxis_title='anchorTs (ms)', yaxis_title='avg net PnL', height=460)
fig

In [6]:
fig = go.Figure()
for col, name in [
    ('cum_avg_flat', 'alwaysFlat'),
    ('cum_avg_long', 'alwaysLong'),
    ('cum_avg_short', 'alwaysShort'),
    ('cum_avg_suggested', 'suggested'),
    ('cum_avg_suggestedIfConf', 'suggestedIfConf'),
]:
    fig.add_trace(go.Scatter(x=wf_clean['anchorTs'], y=wf_clean[col], mode='lines', name=name))
fig.update_layout(title='Cumulative sum of per-anchor expected PnL (proxy curve)', xaxis_title='anchorTs (ms)', yaxis_title='cumulative score', height=460)
fig

In [7]:
fig = px.scatter(wf_clean, x='meanSimilarity', y='avg_suggested', title='Mean similarity vs expected PnL (suggested)')
fig.update_layout(height=380)
fig

## Notes / Next improvements
- If you need a true historical backtest (real future returns after each anchor), we need a price-history source and a policy runner over the real timeline.
- As-is, this notebook is excellent for threshold calibration and drift monitoring using the retrieval distribution.